In [105]:
import pandas as pd
import numpy as np
import sqlite3
import logging
from datetime import datetime

Configure Logging

In [106]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

Extract data from CSV file

In [107]:
def extract_data(file_path):
    try:
        df = pd.read_csv(file_path)
        logging.info(f"Data extracted successfully")
        return df
    except Exception as e:
        logging.error(f"Extraction failed: {e}")
        raise


In [108]:
# Run extract
raw_df = extract_data("ecommerce-purchases.csv")
raw_df.head()

,Address,Lot,AM or PM,Browser Info,Company,Credit Card,CC Exp Date,CC Security Code,CC Provider,Email,Job,IP Address,Language,Purchase Price
0,"16629 Pace Camp Apt. 448\nAlexisborough, NE 77...",46 in,PM,Opera/9.56.(X11; Linux x86_64; sl-SI) Presto/2...,Martinez-Herman,6011929061123406,02/20,900,JCB 16 digit,pdunlap@yahoo.com,"Scientist, product/process development",149.146.147.205,el,98.14
1,"9374 Jasmine Spurs Suite 508\nSouth John, TN 8...",28 rn,PM,Opera/8.93.(Windows 98; Win 9x 4.90; en-US) Pr...,"Fletcher, Richards and Whitaker",3337758169645356,11/18,561,Mastercard,anthony41@reed.com,Drilling engineer,15.160.41.51,fr,70.73
2,Unit 0065 Box 5052\nDPO AP 27450,94 vE,PM,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,"Simpson, Williams and Pham",675957666125,08/19,699,JCB 16 digit,amymiller@morales-harrison.com,Customer service manager,132.207.160.22,de,0.95
3,"7780 Julia Fords\nNew Stacy, WA 45798",36 vm,PM,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0 ...,"Williams, Marshall and Buchanan",6011578504430710,02/24,384,Discover,brent16@olson-robinson.info,Drilling engineer,30.250.74.19,es,78.04
4,"23012 Munoz Drive Suite 337\nNew Cynthia, TX 5...",20 IE,AM,Opera/9.58.(X11; Linux x86_64; it-IT) Presto/2...,"Brown, Watson and Andrews",6011456623207998,10/25,678,Diners Club / Carte Blanche,christopherwright@gmail.com,Fine artist,24.140.33.94,es,77.82


In [109]:
raw_df.shape

(10000, 14)

###Clean and Transform

In [110]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           10000 non-null  object 
 1   Lot               10000 non-null  object 
 2   AM or PM          10000 non-null  object 
 3   Browser Info      10000 non-null  object 
 4   Company           10000 non-null  object 
 5   Credit Card       10000 non-null  int64  
 6   CC Exp Date       10000 non-null  object 
 7   CC Security Code  10000 non-null  int64  
 8   CC Provider       10000 non-null  object 
 9   Email             10000 non-null  object 
 10  Job               10000 non-null  object 
 11  IP Address        10000 non-null  object 
 12  Language          10000 non-null  object 
 13  Purchase Price    10000 non-null  float64
dtypes: float64(1), int64(2), object(11)
memory usage: 1.1+ MB


In [111]:
raw_df.isnull().sum().sum()

np.int64(0)

In [112]:
raw_df.duplicated().sum()

np.int64(0)

In [113]:
df = raw_df.copy()

In [114]:
df.columns

Index(['Address', 'Lot', 'AM or PM', 'Browser Info', 'Company', 'Credit Card',
       'CC Exp Date', 'CC Security Code', 'CC Provider', 'Email', 'Job',
       'IP Address', 'Language', 'Purchase Price'],
      dtype='object')

In [115]:
def transform_data(df):
    df = df.copy()

    # Standardize column names
    df.columns = (df.columns.str.lower().str.replace(" ", "_"))

    # Convert AM or PM to categorical values
    df["am_or_pm"] = df["am_or_pm"].map({"AM": "Morning", "PM": "Evening"})

    # Mask sensitive credit card data
    df["credit_card"] = df["credit_card"].astype(str).str[-4:]

    # Convert purchase price to rounded float
    df["purchase_price"] = df["purchase_price"].round(2)

    # Extract email domain
    df["email_domain"] = df["email"].str.split("@").str[1]

    # Add ingestion timestamp
    df["ingestion_timestamp"] = datetime.now()

    logging.info("Data transformation completed")
    return df

In [116]:
transformed_df = transform_data(df)
transformed_df.head()

,address,lot,am_or_pm,browser_info,company,credit_card,cc_exp_date,cc_security_code,cc_provider,email,job,ip_address,language,purchase_price,email_domain,ingestion_timestamp
0,"16629 Pace Camp Apt. 448\nAlexisborough, NE 77...",46 in,Evening,Opera/9.56.(X11; Linux x86_64; sl-SI) Presto/2...,Martinez-Herman,3406,02/20,900,JCB 16 digit,pdunlap@yahoo.com,"Scientist, product/process development",149.146.147.205,el,98.14,yahoo.com,2026-02-02 15:25:22.368188
1,"9374 Jasmine Spurs Suite 508\nSouth John, TN 8...",28 rn,Evening,Opera/8.93.(Windows 98; Win 9x 4.90; en-US) Pr...,"Fletcher, Richards and Whitaker",5356,11/18,561,Mastercard,anthony41@reed.com,Drilling engineer,15.160.41.51,fr,70.73,reed.com,2026-02-02 15:25:22.368188
2,Unit 0065 Box 5052\nDPO AP 27450,94 vE,Evening,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,"Simpson, Williams and Pham",6125,08/19,699,JCB 16 digit,amymiller@morales-harrison.com,Customer service manager,132.207.160.22,de,0.95,morales-harrison.com,2026-02-02 15:25:22.368188
3,"7780 Julia Fords\nNew Stacy, WA 45798",36 vm,Evening,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0 ...,"Williams, Marshall and Buchanan",0710,02/24,384,Discover,brent16@olson-robinson.info,Drilling engineer,30.250.74.19,es,78.04,olson-robinson.info,2026-02-02 15:25:22.368188
4,"23012 Munoz Drive Suite 337\nNew Cynthia, TX 5...",20 IE,Morning,Opera/9.58.(X11; Linux x86_64; it-IT) Presto/2...,"Brown, Watson and Andrews",7998,10/25,678,Diners Club / Carte Blanche,christopherwright@gmail.com,Fine artist,24.140.33.94,es,77.82,gmail.com,2026-02-02 15:25:22.368188


In [117]:
transformed_df.columns

Index(['address', 'lot', 'am_or_pm', 'browser_info', 'company', 'credit_card',
       'cc_exp_date', 'cc_security_code', 'cc_provider', 'email', 'job',
       'ip_address', 'language', 'purchase_price', 'email_domain',
       'ingestion_timestamp'],
      dtype='object')

###Data Quality Checks

In [118]:
def data_quality_checks(df):
    issues = {}

    issues["null_values"] = df.isnull().sum().sum()
    issues["negative_purchase_price"] = (df["purchase_price"] < 0).sum()
    issues["email_validity"] = df["email"].str.contains("@").sum() == df.shape[0]

    logging.info(f"Data Quality Report: {issues}")
    return issues


In [119]:
dq_report = data_quality_checks(transformed_df)
dq_report

{'null_values': np.int64(0),
 'negative_purchase_price': np.int64(0),
 'email_validity': np.True_}

###Load Phase – Store Clean Data into Database

In [120]:
def load_to_database(df, db_name, table_name):
    try:
        conn = sqlite3.connect(db_name)
        df.to_sql(table_name, conn, if_exists="replace", index=False)
        conn.close()
        logging.info(f"Data loaded into database: {table_name}")
    except Exception as e:
        logging.error(f"Loading failed: {e}")
        raise

In [121]:
load_to_database(
    df,
    db_name="ecommerce.db",
    table_name="clean_purchases"
)

###Validate Loaded Data (POST-LOAD CHECK)

In [122]:
conn = sqlite3.connect("ecommerce.db")
pd.read_sql("SELECT COUNT(*) AS total_records FROM clean_purchases", conn)

,total_records
0,10000


###End-to-End Pipeline Function

In [123]:
def run_etl_pipeline(file_path):
    logging.info("ETL Pipeline Started")

    df = extract_data(file_path)
    df = transform_data(df)
    data_quality_checks(df)
    load_to_database(df, "ecommerce.db", "clean_purchases")

    logging.info("ETL Pipeline Completed Successfully")


In [124]:
run_etl_pipeline("ecommerce-purchases.csv")

In [125]:
summary = {
    "total_records_loaded": transformed_df.shape[0],
    "columns_loaded": transformed_df.shape[1],
    "database": "SQLite",
    "table_name": "clean_purchases"
}

summary

{'total_records_loaded': 10000,
 'columns_loaded': 16,
 'database': 'SQLite',
 'table_name': 'clean_purchases'}